In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 500

### Загрузим датасет с машинами. Цель - верно восстанавливать для каждой из них цену продажи!

In [100]:
data = pd.read_csv('../data/autos.csv')

data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [101]:
### Колонка с тергетом - "selling price"

X = data.drop("selling_price", axis=1)
y = data["selling_price"]

### Будем замерять MSLE!
### Поэтому прологарифмируем таргет
### А после оптимизируем MSE

y = y.apply(np.log1p)

In [102]:
### Разделим выборку на трейн и тест!

from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

__Задание__ 

Реализуйте свой MeanTargetEncoder с добавленем некоторого шума!

Однажды в лекционном материале, обсуждая счетчики, мы говорили с вами о том, что из-за них модели могут переобучаться. Один из способов бороться с этим - валидировать расчеты среднего таргета (стратегия отложенной выборки / расчеты на кросс-валидации). Но есть еще проще!

Можно просто к значению счетчика добавить случайный шум (зашумить данные)!

Напомним, что рассчитываться новые признаки должны по такой формуле:

$$
g_j = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}{l} + C * \epsilon
$$



Пусть шум будет случайной величиной из нормального стандартного распределения, то есть $\epsilon \sim N(0, 1) $, а $ C = 0.006$.

Создавая свой класс-трансформер, наследуйтесь от классов `BaseEstimator, TransformerMixin` из `sklearn.base`. Трансформер не должен модифицировать передаваемую ему выборку inplace, а все необходимые статистики нужно считать только по обучающей выборке в методе `fit`. Ваш трансформер должен принимать при инициализации список из категориальных признаков и список из числовых признаков. 

Если для какого-то признака в тестовой выборке отсутствует значение, трансформер должен поставить там 0.

На выходе должен получиться датасет того же размера с измененными категориальными признаками.

Класс MeanTargetEncoderNoise должен иметь следующую сигнатуру:



In [22]:
from sklearn.base import BaseEstimator, TransformerMixin

class MeanTargetEncoderNoise(BaseEstimator, TransformerMixin):
    
    def __init__(self, categorical, numeric):
        
        ### Your code is here
    
    def fit(self, X, y):

        ### Your code is here

        return self
        
    def transform(self, df):
        
        ### Your code is here
        
        return temp

Разделите колонки на вещественные и категориальные. Приведите все категориальные колонки к типу `object`.

Далее применим наш кодировщик к `X_train, X_test`, так же как например мы применяем `StandardScaler`, чтобы проверить работоспособность нашего класса. Установите зерно датчика случайный чисел `np.random.seed(1)`.

После того, как вы изменили обучающую и тестовую выборки, сохраните первые 10 строк полученного промежуточного датафрейма обучающей выборки (`X_train`) в файл в формате csv с сепаратором `;`. Не забудьте индекс. Отправьте полученный файл в форму ниже.

Список колонок которые должны быть в файле для сдачи:
```py
cols = [
    "km_driven",
    "name",
    "year",
    "fuel",
    "seller_type",
    "transmission",
    "owner"
]
```

### Ваше решение


Разделение колонок на категориальные и числовые.

In [103]:
object_cols = ['name', 'year', 'fuel', 'seller_type', 'transmission', 'owner']
num_cols = ['km_driven']

X.head()

,name,year,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,141000,Diesel,Individual,Manual,Second Owner


Реализация класса MeanTargetEncoderNoise.

In [344]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import KFold
import itertools

class MeanTargetEncoder_(BaseEstimator, TransformerMixin):
    
    def __init__(self, categorical_cols, numeric_cols, target_name, mte_fillna_strategy="mean", mte_strategy="cv_split", noise_k=0.006):              
    
        self.categorical_cols = categorical_cols
        self.numeric_cols = numeric_cols
        self.target_name = target_name
        self.mte_fillna_strategy = mte_fillna_strategy
        self.mte_strategy = mte_strategy
        self.noise_k = noise_k

    def fit(self, X, y):
        
        X_fit = X.copy()
        y_fit = y.copy()
        
        X_with_target = pd.concat((X_fit, y_fit), axis=1) 
        
        ### Посчитаем на валидации средние значения таргета
        if self.mte_strategy == "none":

            self.dict_of_means = {}
            for col in self.categorical_cols:
                self.dict_of_means[col] = X_with_target.groupby(col)[self.target_name].mean()


        elif self.mte_strategy == "noise":

            self.dict_of_means = {}
            for col in self.categorical_cols:
                self.dict_of_means[col] = X_with_target.groupby(col)[self.target_name].mean() + self.noise_k * np.random.randn(len(X_with_target[col].unique()))
    

        elif self.mte_strategy == "cv_split":

            kf = KFold(n_splits=5, shuffle=True, random_state=42)
            self.dict_of_means = {}
            for col in self.categorical_cols:
                col_means = {}
                for index, (train_index, val_index) in enumerate(kf.split(X_with_target)):

                    kf_train, kf_val = X_with_target.iloc[train_index], X_with_target.iloc[val_index]
                    ### посчитали на ~4ех трейновых сплитах средние значения таргета для каждой категории
                    count_map = kf_train.groupby(col)[self.target_name].mean()
                    count_map = count_map + self.noise_k * np.random.randn(len(count_map))
                    col_means[f"fold_{index}"] = count_map

                self.dict_of_means[col] = pd.concat(col_means.values(), axis=1).mean(axis=1)


        elif self.mte_strategy == "multiclass":

            ### Запомним все классы
            self.mte_class_names = {col : sorted([f"{col}_{value}" for value in X_with_target[self.target_name].unique()]) for col in self.categorical_cols}


        else:
            raise ValueError("Unsupported mte_strategy strategy. Please choose either 'none', 'noise' or 'cv_split'.")
        
        return self
        

    def transform(self, X, y):
        
        X_fit = X.copy()
        y_fit = y.copy()
        
        df_ = pd.concat((X_fit, y_fit), axis=1) 

        if self.mte_strategy != "multiclass":

            for col in self.categorical_cols:
                df_[col] = df_[col].map(self.dict_of_means[col])
                
                ### Если на проде или на тестовой выборке мы получили новые категории, которые не были в обучении
                ### то заполняем эти данные по общему среднему всех полученных значений для того чтобы избавится от NaN-ов
                if df_[col].isna().sum() != 0:
                    if self.mte_fillna_strategy == 'mean':
                        mean_value = self.dict_of_means[col].values.mean()
                        df_[col] = df_[col].fillna(mean_value)
                    elif self.mte_fillna_strategy == 'mode':
                        mode_value = df_[col].mode()[0]
                        df_[col] = df_[col].fillna(mode_value)
                    else:
                        raise ValueError("Unsupported fillna strategy. Please choose either 'mean' or 'mode'.")

        else:

            target_dummies_cols = [] # Сюда будем записывать все столбцы, которые будут закодированы 

            for col in self.categorical_cols:

                # Создаем фиктивные переменные для целевой переменной
                target_dummies = pd.get_dummies(df_[self.target_name], prefix=col)
            
                ### Фиксируем полученные столбцы после кодирования
                target_dummies_cols.extend(target_dummies.columns)

                # Объединяем DataFrame с фиктивными переменными
                df_ = pd.concat((df_, target_dummies), axis=1)

                # Для каждой фиктивной переменной вычисляем среднее значение в каждой категории
                for tg in target_dummies.columns:
                    df_[tg] = df_.groupby(col)[tg].transform("mean")

                # Удаляем из итогового DataFrame исходный категориальный признак
                df_ = df_.drop(col, axis=1)

            # Получение списка всех имен столбцов по классам
            all_mte = list(itertools.chain(*list(self.mte_class_names.values())))

            ### На проде или на тесте мы можем не получить классы, которые были в обучении,
            ### в таком случае нам необходимо зафиксировать данные классы и создать столбцы для 
            ### данных классов, заполнив их нулями, таким образом мы сохраним изначальную размерность датасета на обучении
            missing_columns = [
                x 
                for x in all_mte
                if x not in df_.columns
                and
                x not in self.numeric_cols
            ]

            ### Отсутствующие классы
            ### необходимо добавить: заполним их просто нулями
            
            if len(missing_columns) != 0:

                zeros = np.zeros((df_.shape[0], len(missing_columns)))
                zeros = pd.DataFrame(zeros,
                                    columns=missing_columns,
                                    index=df_.index)

                df_ = pd.concat((df_, zeros), axis=1)


            ### Также на проде или на тесте мы можем получить новые классы, которые не видлеи на обучении,
            ### таким образом мы можем получить новые столбцы, делать этого нельзя, так как мы потеряем нашу размерноть, 
            ### поэтому фиксируем их и отбрасываем
            extra_columns = [
                x
                for x in target_dummies_cols
                if x not in all_mte
            ]

            ### Новые классы необходимо убрать
            df_ = df_.drop(extra_columns, axis=1)


        return df_[sorted(df_.columns)]

In [312]:
import itertools
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

class OneHotEncoder_(BaseEstimator, TransformerMixin):
    
    def __init__(self, categorical_cols, numeric_cols, target_name, encoding_method="one_hot"):              
    
        self.categorical_cols = categorical_cols
        self.numeric_cols = numeric_cols
        self.target_name = target_name
        self.encoding_method = encoding_method
        self.label_encoding_dicts = {}  # Добавление атрибута для хранения словарей кодирования

    def fit(self, X, y):
        
        X_fit = X.copy()
        y_fit = y.copy()
        
        X_with_target = pd.concat((X_fit, y_fit), axis=1) 
        
        if self.encoding_method == 'one_hot':

            ### Запомним все ohe колонки и их названия
            self.ohe_names = {col : sorted([f"{col}_{value}" for value in X_with_target[col].unique()]) for col in self.categorical_cols}


        elif self.encoding_method == 'label':
            ### Закодируем каждое категориальное значение в категориальном признаке  
            for col in self.categorical_cols:
                label_encoding_dicts = {val: idx for idx, val in enumerate(X_with_target[col].unique())}
                self.label_encoding_dicts[col] = label_encoding_dicts


        return self
        
    def transform(self, X, y):
        
        X_fit = X.copy()
        y_fit = y.copy()
        
        df_ = pd.concat((X_fit, y_fit), axis=1) 

        if self.encoding_method == 'one_hot':
            data_part = pd.get_dummies(df_[self.categorical_cols], prefix=None)
            data_part.replace({False: 0, True: 1}, inplace=True)

            # Дропаем все оригинальные категориальные колонки, так как они нам больше не нужны 
            df_ = df_.drop(self.categorical_cols, axis=1)
            # Присоединяем все преобразованные категориальные колонки к датасету 
            df_ = pd.concat((df_, data_part), axis=1)

            # Получение списка всех имен столбцов one-hot в рамках данного класса 
            all_ohe = list(itertools.chain(*list(self.ohe_names.values())))

            ### Фиксируем полученные столбцы после кодирования ohe
            data_part_cols = data_part.columns

            ### На проде или на тесте мы можем не получить категории, которые были в обучении,
            ### в таком случае нам необходимо зафиксировать данные категории и создать столбцы для 
            ### данных категорий, заполнив их нулями, таким образом мы сохраним изначальную размерность датасета на обучении
            missing_columns = [
                x 
                for x in all_ohe
                if x not in df_.columns
                and
                x not in self.numeric_cols
            ]

            ### Отсутствующие категории (бинарные колонки)
            ### необходимо добавить: заполним их просто нулями
            
            if len(missing_columns) != 0:

                zeros = np.zeros((df_.shape[0], len(missing_columns)))
                zeros = pd.DataFrame(zeros,
                                    columns=missing_columns,
                                    index=df_.index)

                df_ = pd.concat((df_, zeros), axis=1)


            ### Также на проде или на тесте мы можем получить новые категории, которые не видлеи на обучении,
            ### таким образом мы можем получить новые столбцы, делать этого нельзя, так как мы потеряем нашу размерноть, 
            ### поэтому фиксируем их и отбрасываем
            extra_columns = [
                x
                for x in data_part_cols
                if x not in all_ohe
            ]

            ### Новые категории необходимо убрать
            df_ = df_.drop(extra_columns, axis=1)

            # Инициализируем новый DataFrame для хранения результата
            new_df = pd.DataFrame()

            # Удаляем первый столбец для каждой категории для избегания мультиколлинеарности 
            for unique_pref in self.ohe_names.keys():
                filtered_cols = df_.filter(like=unique_pref).columns
                # Получаем все столбцы категории
                filtered_df = df_[filtered_cols]
                # Удаляем первый столбец и добавляем остальные в новый DataFrame
                new_df = pd.concat([new_df, filtered_df.drop(filtered_df.columns[0], axis=1)], axis=1)

            # Обновляем df_ новыми значениями без первого столбца для каждой категории
            df_ = new_df

        elif self.encoding_method == 'label':

            for col in self.categorical_cols:
                encoding_dict = self.label_encoding_dicts[col]
                df_[col] = df_[col].map(encoding_dict)
                ### Если на проде или на тестовой выборке мы получили новые категории,
                # то заполняем неизвестны категории значением -1
                if df_[col].isna().sum() != 0:
                    df_[col] = df_[col].fillna(-1)

        else:
            raise ValueError("Unsupported encoding method. Please choose either 'one_hot' or 'label'.")
        
            
        return df_[sorted(df_.columns)]


In [345]:
transformer = MeanTargetEncoder_(categorical_cols=object_cols, numeric_cols=num_cols, target_name="selling_price", mte_strategy="multiclass")

transformer.fit(X_train, y_train)

train = transformer.transform(X_train, y_train)
test = transformer.transform(X_test, y_test)

train.head(10)

,fuel_10.463131911491967,fuel_10.53212287826962,fuel_10.596659732783579,fuel_10.645448706505872,fuel_10.714439990727769,fuel_10.778977123006351,fuel_10.819798284210286,fuel_10.841774582282179,fuel_10.859018228147887,fuel_10.915106645867503,fuel_10.933124826700707,fuel_10.968215530759236,fuel_11.002116507732017,fuel_11.0186455364637,fuel_11.034905792929415,fuel_11.082157933374816,fuel_11.127277689932466,fuel_11.141876276228,fuel_11.156264806643742,fuel_11.170449240431308,fuel_11.184435286790631,fuel_11.19822841866684,fuel_11.225256725762893,fuel_11.264476926102367,fuel_11.273309485087486,fuel_11.277215789596893,fuel_11.289794413577894,fuel_11.302216779257382,fuel_11.314486721293981,fuel_11.350418300109132,fuel_11.385103457032141,fuel_11.407564949312402,fuel_11.407576060361786,fuel_11.438212694578302,fuel_11.440365524765756,fuel_11.461642696843066,fuel_11.48247656671073,fuel_11.49273296168228,fuel_11.502885230075814,fuel_11.51293546492023,fuel_11.561725152903833,fuel_11.571203807011969,fuel_11.608244735642321,fuel_11.6172944892629,fuel_11.652687407345388,fuel_11.661345470088502,fuel_11.695255355062795,fuel_11.703554089007538,fuel_11.719947764402807,fuel_11.736077016252437,fuel_11.744045122410057,fuel_11.775297421715827,fuel_11.798111925972618,fuel_11.80560254159177,fuel_11.813037464800539,fuel_11.835016210489899,fuel_11.849404844423074,fuel_11.863589378812122,fuel_11.884495917930655,fuel_11.904974309480183,fuel_11.91170829631447,fuel_11.918397239722838,fuel_11.92504173829169,fuel_11.951186847493476,fuel_11.976665770490767,fuel_11.982935344196433,fuel_12.001511614739405,fuel_12.00762780434872,fuel_12.013706813470414,fuel_12.04355959836804,fuel_12.06105265381003,fuel_12.07254696717504,fuel_12.08954444723597,fuel_12.100717685412471,fuel_12.111767460549332,fuel_12.117246896289503,fuel_12.128116509451258,fuel_12.154784614286667,fuel_12.170450649220749,fuel_12.180759965737863,fuel_12.201065128819632,fuel_12.206077645517674,fuel_12.211060187041213,fuel_12.230765258120545,fuel_12.25486757159303,fuel_12.259618151783465,fuel_12.264346270624157,fuel_12.269052139516223,fuel_12.278397958261774,fuel_12.28765724080691,fuel_12.296831574998258,fuel_12.301387370778713,fuel_12.30592250537653,fuel_12.310437165348775,fuel_12.323860125621126,fuel_12.33270970174018,fuel_12.345834587905333,fuel_12.354492650648448,fuel_12.358798024384122,fuel_12.367345048436391,fuel_12.380030154325457,fuel_12.384223015005313,fuel_12.388398368982115,fuel_12.409017571151187,fuel_12.421188057397055,fuel_12.425212191503038,fuel_12.429220196836383,fuel_12.43320821811392,fuel_12.449018824140563,fuel_12.45293662970407,fuel_12.468440756144114,fuel_12.487488878546145,fuel_12.502470376049185,fuel_12.506180941677357,fuel_12.509877789891931,fuel_12.524530013005734,fuel_12.531776395271262,fuel_12.54254845357358,fuel_12.546113507028412,fuel_12.549665896013254,fuel_12.560247968016562,fuel_12.567240979056246,fuel_12.570719231334277,fuel_12.577639650232573,fuel_12.581081982573842,fuel_12.59473402514672,fuel_12.608202196848836,fuel_12.611541086966115,fuel_12.618185607609803,fuel_12.628070334272698,fuel_12.644330802262578,fuel_12.647551406590285,fuel_12.660331092405906,fuel_12.676079399771027,fuel_12.688501881189456,fuel_12.691583538230217,fuel_12.706850963741102,fuel_12.721888795877375,fuel_12.73670383776449,fuel_12.739640688812674,fuel_12.75130259456002,fuel_12.754196944209095,fuel_12.757082940769225,fuel_12.762830066509972,fuel_12.765691290604371,fuel_12.768544351446794,fuel_12.77138929548529,fuel_12.779875885354993,fuel_12.791080452981857,fuel_12.793862088206213,fuel_12.80216086806413,fuel_12.807655372286902,fuel_12.81584093453868,fuel_12.821260987319457,fuel_12.82396003700373,fuel_12.834683971615659,fuel_12.840003118946523,fuel_12.847929163278053,fuel_12.850557278777309,fuel_12.861001210669146,fuel_12.873904582205107,fuel_12.886643575525625,fuel_12.889172015485954,fuel_12.896719202134523,fuel_12.899222326086994,fuel_12.901719200051183,fuel_12.92391243868049,fuel_12.926350926565435,fu

In [335]:
test.head(10)

,fuel_10.308985993422082,fuel_10.463131911491967,fuel_10.53212287826962,fuel_10.596659732783579,fuel_10.645448706505872,fuel_10.668978650219241,fuel_10.714439990727769,fuel_10.778977123006351,fuel_10.819798284210286,fuel_10.841774582282179,fuel_10.859018228147887,fuel_10.915106645867503,fuel_10.933124826700707,fuel_10.950824090522445,fuel_10.968215530759236,fuel_11.002116507732017,fuel_11.0186455364637,fuel_11.034905792929415,fuel_11.082157933374816,fuel_11.127277689932466,fuel_11.141876276228,fuel_11.156264806643742,fuel_11.170449240431308,fuel_11.184435286790631,fuel_11.19822841866684,fuel_11.225256725762893,fuel_11.264476926102367,fuel_11.273309485087486,fuel_11.277215789596893,fuel_11.289794413577894,fuel_11.302216779257382,fuel_11.314486721293981,fuel_11.350418300109132,fuel_11.385103457032141,fuel_11.407564949312402,fuel_11.407576060361786,fuel_11.420821140914589,fuel_11.438212694578302,fuel_11.440365524765756,fuel_11.461642696843066,fuel_11.48247656671073,fuel_11.49273296168228,fuel_11.502885230075814,fuel_11.51293546492023,fuel_11.561725152903833,fuel_11.571203807011969,fuel_11.608244735642321,fuel_11.6172944892629,fuel_11.652687407345388,fuel_11.661345470088502,fuel_11.695255355062795,fuel_11.703554089007538,fuel_11.719947764402807,fuel_11.736077016252437,fuel_11.744045122410057,fuel_11.775297421715827,fuel_11.798111925972618,fuel_11.80560254159177,fuel_11.813037464800539,fuel_11.835016210489899,fuel_11.849404844423074,fuel_11.863589378812122,fuel_11.884495917930655,fuel_11.898194668458823,fuel_11.904974309480183,fuel_11.91170829631447,fuel_11.918397239722838,fuel_11.92504173829169,fuel_11.951186847493476,fuel_11.976665770490767,fuel_11.982935344196433,fuel_12.001511614739405,fuel_12.00762780434872,fuel_12.013706813470414,fuel_12.04355959836804,fuel_12.06105265381003,fuel_12.07254696717504,fuel_12.08954444723597,fuel_12.100717685412471,fuel_12.111767460549332,fuel_12.117246896289503,fuel_12.128116509451258,fuel_12.154784614286667,fuel_12.170450649220749,fuel_12.180759965737863,fuel_12.19602736016897,fuel_12.201065128819632,fuel_12.206077645517674,fuel_12.211060187041213,fuel_12.230765258120545,fuel_12.25486757159303,fuel_12.259618151783465,fuel_12.264346270624157,fuel_12.269052139516223,fuel_12.278397958261774,fuel_12.28765724080691,fuel_12.296831574998258,fuel_12.301387370778713,fuel_12.30592250537653,fuel_12.310437165348775,fuel_12.323860125621126,fuel_12.33270970174018,fuel_12.345834587905333,fuel_12.354492650648448,fuel_12.358798024384122,fuel_12.367345048436391,fuel_12.380030154325457,fuel_12.384223015005313,fuel_12.388398368982115,fuel_12.409017571151187,fuel_12.421188057397055,fuel_12.425212191503038,fuel_12.429220196836383,fuel_12.43320821811392,fuel_12.449018824140563,fuel_12.45293662970407,fuel_12.468440756144114,fuel_12.479913113434042,fuel_12.487488878546145,fuel_12.502470376049185,fuel_12.506180941677357,fuel_12.509877789891931,fuel_12.524530013005734,fuel_12.531776395271262,fuel_12.54254845357358,fuel_12.546113507028412,fuel_12.549665896013254,fuel_12.556733038263905,fuel_12.560247968016562,fuel_12.567240979056246,fuel_12.570719231334277,fuel_12.577639650232573,fuel_12.581081982573842,fuel_12.59473402514672,fuel_12.608202196848836,fuel_12.611541086966115,fuel_12.618185607609803,fuel_12.628070334272698,fuel_12.644330802262578,fuel_12.647551406590285,fuel_12.650761671914688,fuel_12.660331092405906,fuel_12.676079399771027,fuel_12.679199517372098,fuel_12.688501881189456,fuel_12.691583538230217,fuel_12.706850963741102,fuel_12.721888795877375,fuel_12.73670383776449,fuel_12.739640688812674,fuel_12.75130259456002,fuel_12.754196944209095,fuel_12.757082940769225,fuel_12.762830066509972,fuel_12.765691290604371,fuel_12.768544351446794,fuel_12.77138929548529,fuel_12.779875885354993,fuel_12.791080452981857,fuel_12.793862088206213,fuel_12.80216086806413,fuel_12.807655372286902,fuel_12.81584093453868,fuel_12.821260987319457,fuel_12.82396003700373,fuel_12.834683971615659,fuel_12.840003118946523,fuel_12.847929163278053,fu

In [313]:
transformer = OneHotEncoder_(categorical_cols=object_cols, numeric_cols=num_cols, target_name="selling_price", encoding_method="one_hot")

transformer.fit(X_train, y_train)

['name_Ambassador CLASSIC 1500 DSL AC',
 'name_Ambassador Grand 1800 ISZ MPFI PW CL',
 'name_Audi A4 1.8 TFSI',
 'name_Audi A4 2.0 TDI',
 'name_Audi A4 2.0 TDI 177 Bhp Premium Plus',
 'name_Audi A4 3.0 TDI Quattro',
 'name_Audi A4 30 TFSI Technology',
 'name_Audi A4 35 TDI Premium Plus',
 'name_Audi A4 New  2.0 TDI Multitronic',
 'name_Audi A5 Sportback',
 'name_Audi A6 2.0 TDI  Design Edition',
 'name_Audi A6 2.0 TDI Premium Plus',
 'name_Audi A6 2.7 TDI',
 'name_Audi A6 2.8 FSI',
 'name_Audi A8 4.2 TDI',
 'name_Audi A8 L 3.0 TDI quattro',
 'name_Audi Q3 2.0 TDI Quattro Premium Plus',
 'name_Audi Q3 35 TDI Quattro Technology',
 'name_Audi Q5 2.0 TDI',
 'name_Audi Q5 2.0 TFSI Quattro',
 'name_Audi Q5 2.0 TFSI Quattro Premium Plus',
 'name_Audi Q5 3.0 TDI Quattro Technology',
 'name_Audi RS7 2015-2019 Sportback Performance',
 'name_BMW 3 Series 320d Luxury Line',
 'name_BMW 3 Series 320d Sport',
 'name_BMW 3 Series 320d Sport Line',
 'name_BMW 3 Series GT Luxury Line',
 'name_BMW 5 Seri

In [134]:
X_train.describe(include=object)

,name,fuel,seller_type,transmission,owner
count,3472,3472,3472,3472,3472
unique,1331,5,3,2,5
top,Maruti Swift Dzire VDI,Petrol,Individual,Manual,First Owner
freq,54,1724,2600,3091,2255


In [175]:
transformer = OneHotEncoder_(categorical_cols=object_cols, numeric_cols=num_cols, target_name="selling_price", encoding_method="one_hot")

transformer.fit(X_train, y_train)

train = transformer.transform(X_train)
test = transformer.transform(X_test)

train.head(10)

C:\Users\danil\AppData\Local\Temp\ipykernel_2828\521968890.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_part.replace({False: 0, True: 1}, inplace=True)


dict_keys(['name', 'year', 'fuel', 'seller_type', 'transmission', 'owner'])
Удалил: name_Ambassador CLASSIC 1500 DSL AC
Удалил: year_1992
Удалил: fuel_CNG
Удалил: seller_type_Dealer
Удалил: transmission_Automatic
Удалил: owner_First Owner


C:\Users\danil\AppData\Local\Temp\ipykernel_2828\521968890.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_part.replace({False: 0, True: 1}, inplace=True)


dict_keys(['name', 'year', 'fuel', 'seller_type', 'transmission', 'owner'])
Удалил: name_Ambassador CLASSIC 1500 DSL AC
Удалил: year_1992
Удалил: fuel_CNG
Удалил: seller_type_Dealer
Удалил: transmission_Automatic
Удалил: owner_First Owner


,fuel_Diesel,fuel_Electric,fuel_LPG,fuel_Petrol,name_Ambassador Grand 1800 ISZ MPFI PW CL,name_Audi A4 1.8 TFSI,name_Audi A4 2.0 TDI,name_Audi A4 2.0 TDI 177 Bhp Premium Plus,name_Audi A4 3.0 TDI Quattro,name_Audi A4 30 TFSI Technology,name_Audi A4 35 TDI Premium Plus,name_Audi A4 New 2.0 TDI Multitronic,name_Audi A5 Sportback,name_Audi A6 2.0 TDI Design Edition,name_Audi A6 2.0 TDI Premium Plus,name_Audi A6 2.7 TDI,name_Audi A6 2.8 FSI,name_Audi A8 4.2 TDI,name_Audi A8 L 3.0 TDI quattro,name_Audi Q3 2.0 TDI Quattro Premium Plus,name_Audi Q3 35 TDI Quattro Technology,name_Audi Q5 2.0 TDI,name_Audi Q5 2.0 TFSI Quattro,name_Audi Q5 2.0 TFSI Quattro Premium Plus,name_Audi Q5 3.0 TDI Quattro Technology,name_Audi RS7 2015-2019 Sportback Performance,name_BMW 3 Series 320d Luxury Line,name_BMW 3 Series 320d Sport,name_BMW 3 Series 320d Sport Line,name_BMW 3 Series GT Luxury Line,name_BMW 5 Series 520d Luxury Line,name_BMW 5 Series 525d Sedan,name_BMW 5 Series 530i,name_BMW 7 Series 730Ld,name_BMW 7 Series Signature 730Ld,name_BMW X1 sDrive 20d Exclusive,name_BMW X1 sDrive 20d xLine,name_BMW X1 sDrive20d,name_BMW X1 sDrive20d M Sport,name_BMW X5 xDrive 30d xLine,name_Chevrolet Aveo 1.4,name_Chevrolet Aveo 1.4 LT BSIV,name_Chevrolet Aveo 1.6 LT with ABS,name_Chevrolet Aveo U-VA 1.2,name_Chevrolet Aveo U-VA 1.2 LS,name_Chevrolet Aveo U-VA 1.2 LT,name_Chevrolet Aveo U-VA 1.2 LT WO ABS Airbag,name_Chevrolet Beat Diesel,name_Chevrolet Beat Diesel LS,name_Chevrolet Beat Diesel LT,name_Chevrolet Beat Diesel LT Option,name_Chevrolet Beat Diesel PS,name_Chevrolet Beat LS,name_Chevrolet Beat LT,name_Chevrolet Beat LT LPG,name_Chevrolet Beat LT Option,name_Chevrolet Beat PS,name_Chevrolet Captiva LT,name_Chevrolet Cruze LT,name_Chevrolet Cruze LTZ,name_Chevrolet Cruze LTZ AT,name_Chevrolet Enjoy 1.3 TCDi LS 8,name_Chevrolet Enjoy TCDi LT 7 Seater,name_Chevrolet Enjoy TCDi LT 8 Seater,name_Chevrolet Optra 1.6,name_Chevrolet Optra 1.6 LS,name_Chevrolet Optra Magnum 2.0 LS,name_Chevrolet Optra Magnum 2.0 LS BSIII,name_Chevrolet Optra Magnum 2.0 LT,name_Chevrolet Sail 1.2 Base,name_Chevrolet Sail 1.2 LT ABS,name_Chevrolet Sail 1.3 LS,name_Chevrolet Sail Hatchback 1.2 LS,name_Chevrolet Sail Hatchback 1.3 TCDi,name_Chevrolet Sail Hatchback LS ABS,name_Chevrolet Sail Hatchback LT ABS,name_Chevrolet Sail LS ABS,name_Chevrolet Spark 1.0,name_Chevrolet Spark 1.0 LS,name_Chevrolet Spark 1.0 LT,name_Chevrolet Spark 1.0 LT BS3,name_Chevrolet Spark 1.0 LT Option Pack w/ Airbag,name_Chevrolet Spark 1.0 PS,name_Chevrolet Tavera LS B3 7 Seats BSII,name_Chevrolet Tavera LT L1 7 Seats BSIII,name_Chevrolet Tavera Neo 2 LS B4 7 Str BSIII,name_Chevrolet Tavera Neo 2 LT L 9 Str,name_Chevrolet Tavera Neo 3 10 Seats BSIV,name_Chevrolet Tavera Neo 3 9 Str BSIII,name_Chevrolet Tavera Neo 3 LS 7 C BSIII,name_Chevrolet Tavera Neo 3 LT 9 Seats BSIII,name_Chevrolet Tavera Neo 3 Max 9 Str BSIII,name_Chevrolet Tavera Neo LS B3 - 7(C) seats BSIII,name_Daewoo Matiz SD,name_Datsun GO Plus A,name_Datsun GO Plus A Option Petrol,name_Datsun GO Plus Remix Limited Edition,name_Datsun GO Plus T,name_Datsun GO Plus T BSIV,name_Datsun GO Plus T Option,name_Datsun GO Plus T Option BSIV,name_Datsun GO Plus T Option Petrol,name_Datsun GO T BSIV,name_Datsun GO T Option BSIV,name_Datsun GO T Petrol,name_Datsun RediGO 1.0 S,name_Datsun RediGO AMT 1.0 S,name_Datsun RediGO SV 1.0,name_Datsun RediGO T Option,name_Datsun redi-GO AMT 1.0 T Option,name_Fiat 500 Lounge,name_Fiat Avventura MULTIJET Emotion,name_Fiat Avventura Urban Cross 1.3 Multijet Emotion,name_Fiat Grande Punto 1.3 Dynamic (Diesel),name_Fiat Grande Punto Active (Diesel),name_Fiat Grande Punto EVO 1.3 Active,name_Fiat Grande Punto EVO 1.3 Dynamic,name_Fiat Grande Punto Emotion 90Hp,name_Fiat Linea 1.3 Emotion,name_Fiat Linea 1.3 Multijet Emotion,name_Fiat Linea Dynamic,name_Fiat Linea Emotion,name_Fiat Linea Emotion (Diesel),name_Fiat Linea T Jet Emotion,name_Fiat Linea T Jet Plus,name_Fiat Palio 1.2 Sport,name_Fiat Palio D 1.9 EL PS

In [297]:
transformer = MeanTargetEncoder_(categorical_cols=object_cols, numeric_cols=num_cols, target_name="selling_price", mte_strategy="none")

transformer.fit(X_train, y_train)

train = transformer.transform(X_train)
test = transformer.transform(X_test)

train.head(10)

,fuel,km_driven,name,owner,seller_type,transmission,year
3294,13.089123,50000,13.469119,12.972935,12.618096,13.771251,13.439563
2290,12.453348,70000,12.115454,12.972935,12.618096,13.771251,11.910155
874,12.453348,50000,12.301762,12.972935,12.618096,12.638189,13.336728
1907,12.453348,92198,12.484095,12.460189,13.147319,12.638189,13.057348
3244,12.453348,3240,12.391951,12.460189,12.618096,12.638189,12.860755
1089,12.453348,10000,12.690560,12.972935,13.147319,12.638189,13.439563
3902,12.453348,90000,11.696890,11.870401,12.618096,12.638189,11.503453
2215,12.453348,79000,11.121822,12.460189,12.618096,12.638189,11.503453
3862,13.089123,99700,13.171979,12.972935,12.618096,12.638189,13.336728
705,13.089123,124000,12.999160,12.460189,12.618096,12.638189,12.245474


In [328]:
transformer = MeanTargetEncoder_(categorical_cols=object_cols, numeric_cols=num_cols, target_name="selling_price", mte_strategy="cv_split")

transformer.fit(X_train, y_train)

train = transformer.transform(X_train, y_train)
test = transformer.transform(X_test, y_test)

train.head(10)

,fuel,km_driven,name,owner,seller_type,selling_price,transmission,year
3294,13.086463,50000,13.469386,12.972850,12.617075,13.422469,13.772506,13.441305
2290,12.456914,70000,12.085621,12.972850,12.617075,11.849405,13.772506,11.908150
874,12.456914,50000,12.298372,12.972850,12.617075,11.982935,12.639052,13.334341
1907,12.456914,92198,12.490197,12.463531,13.146936,12.323860,12.639052,13.056978
3244,12.456914,3240,12.391520,12.463531,12.617075,12.542548,12.639052,12.862057
1089,12.456914,10000,12.694343,12.972850,13.146936,12.821261,12.639052,13.441305
3902,12.456914,90000,11.695774,11.872777,12.617075,11.156265,12.639052,11.507280
2215,12.456914,79000,11.125026,12.463531,12.617075,11.561725,12.639052,11.507280
3862,13.086463,99700,13.167917,12.972850,12.617075,13.304687,12.639052,13.334341
705,13.086463,124000,13.002850,12.463531,12.617075,12.706851,12.639052,12.244315


Проверка работы трансформера.

In [293]:
np.random.seed(1)
transformer = MeanTargetEncoder_(categorical_cols=object_cols, numeric_cols=num_cols, target_name="selling_price", mte_strategy="noise")

transformer.fit(X_train, y_train)

train = transformer.transform(X_train)
test = transformer.transform(X_test)

train.head(10)

,fuel,km_driven,name,owner,seller_type,transmission,year
3294,13.093756,50000,13.483692,12.977730,12.615863,13.771135,13.436559
2290,12.453832,70000,12.117029,12.977730,12.615863,13.771135,11.903115
874,12.453832,50000,12.310456,12.977730,12.615863,12.639805,13.328864
1907,12.453832,92198,12.491443,12.463313,13.152824,12.639805,13.042359
3244,12.453832,3240,12.390906,12.463313,12.615863,12.639805,12.870886
1089,12.453832,10000,12.687432,12.977730,13.152824,12.639805,13.436559
3902,12.453832,90000,11.698702,11.878390,12.615863,12.639805,11.503504
2215,12.453832,79000,11.120678,12.463313,12.615863,12.639805,11.503504
3862,13.093756,99700,13.172879,12.977730,12.615863,12.639805,13.328864
705,13.093756,124000,13.004194,12.463313,12.615863,12.639805,12.241213


In [294]:
display(test.shape)
display(train.shape)

(868, 7)

(3472, 7)

In [49]:
cols = [
    "km_driven",
    "name",
    "year",
    "fuel",
    "seller_type",
    "transmission",
    "owner"
]

In [295]:
train[cols].head(10).to_csv("../data/train.csv", sep=';', index=False)
some = pd.read_csv("../data/train.csv", sep=';')
some

,km_driven,name,year,fuel,seller_type,transmission,owner
0,50000,13.483692,13.436559,13.093756,12.615863,13.771135,12.977730
1,70000,12.117029,11.903115,12.453832,12.615863,13.771135,12.977730
2,50000,12.310456,13.328864,12.453832,12.615863,12.639805,12.977730
3,92198,12.491443,13.042359,12.453832,13.152824,12.639805,12.463313
4,3240,12.390906,12.870886,12.453832,12.615863,12.639805,12.463313
5,10000,12.687432,13.436559,12.453832,13.152824,12.639805,12.977730
6,90000,11.698702,11.503504,12.453832,12.615863,12.639805,11.878390
7,79000,11.120678,11.503504,12.453832,12.615863,12.639805,12.463313
8,99700,13.172879,13.328864,13.093756,12.615863,12.639805,12.977730
9,124000,13.004194,12.241213,13.093756,12.615863,12.639805,12.463313


Обучите несколько деревьев, перебирая максимальную глубину алгоритма из списка `max_depth_list`, а остальные параметры оставьте дефолтными. Выведите лучшее значение гиперпараметра. Постройте график зависимости MSLE на тестовой выборке от значения гиперпараметра. Воспользуйтесь `Pipeline` без `GridSearch`. Проделайте то же самое с `min_samples_split`, `min_impurity_decrease`, `max_leaf_nodes`. (по 2б на каждый параметр)

In [ ]:
max_depth_list = [3, 5, 8, 12]
min_samples_split_list = [10, 50, 100, 500]
min_impurity_decrease_list = [0, 0.1, 0.15, 0.2]
max_leaf_nodes_list = [100, 200, 500]

In [ ]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline

np.random.seed(1)

### Your code is here





Подберите лучшую комбинацию параметров, используя `GridSearchCV` и набор массивов значений параметров из предыдущего задания. Для лучшей комбинации посчитайте MSLE на тестовой выборке. Получились ли лучшие параметры такими же, как если бы вы подбирали их по-отдельности при остальных гиперпараметрах по умолчанию (предыдущее задание)? (2б)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "decision_tree__max_depth": [3, 5, 8, 12],
    "decision_tree__min_samples_split": [10, 50, 100, 500],
    "decision_tree__min_impurity_decrease": [0, 0.1, 0.15, 0.2],
    "decision_tree__max_leaf_nodes": [100, 200, 500]
}
np.random.seed(1)

### Your code is here



